# Analisis Salarios Colombia

por: Jose R. Zapata - https://joserzapata.github.io/

Link: https://joserzapata.github.io/post/


En base a los cambios que se han presentado en la reforma tributaria en Colombia en el 2022 y como se ven afectados los salarios de los trabajadores, se realiza un análisis de los salarios en Colombia para entender cuanto se le descuenta a un trabajador y cuanto seria el dinero que recibe en su bolsillo.


## Descuentos en salarios en Colombia

Para conocer cuanto se le descuenta a un trabajador en Colombia es necesario  tener en cuenta los pagos por:
 - Salud (4%)
 - Pension (4%)
 - ARL (0.5% pero varia segun el riesgo)
 - fondo de solidaridad pensional (variable)
 - Retencion en la fuente (variable) [Articulo 383 del estatuto tributario](https://estatuto.co/383)

Los calculos son diferentes para asalariados que reciben el [salario integral](https://loggro.com/blog/articulo/como-calcular-el-salario-integral-loggro-nomina/)

la tabla de retención en la fuente que esta en base a los valores de UVT (Unidad de Valor Tributario) y que se actualiza cada año. 

### Porcentaje de retencion en la fuente

los rango estan representados en UVT (2023) = $42.412 pesos

 | Rangos en UVT  | Tarifa marginal | retención en la fuente |
| -------------   | --------------- | ----------------------- |
| 0 a 95          | 0%  |                                     |
| \>95 a 150      | 19% | (Ingreso laboral gravado expresado en UVT menos 95 UVT)\*19%               |
| \>150 a 360     | 28% | (Ingreso laboral gravado expresado en UVT menos 150 UVT)\*28% más 10 UVT |
| \>360 a  640    | 33% | (Ingreso laboral gravado expresado en UVT menos 360 UVT)\*33% más 69 UVT   |
| \>640 a  945    | 35% | (Ingreso laboral gravado expresado en UVT menos 640 UVT)\*35% más 162 UVT  |
| \>945 a  2300   | 37% | (Ingreso laboral gravado expresado en UVT menos 945 UVT)\*37% más 268 UVT  |
| \>2300 En adelante |  39%| (Ingreso laboral gravado expresado en UVT menos 2300 UVT)\*39% más 770 UVT |

Nota: Sera unicamente para salarios y no se tiene en cuenta ingresos extra. Ademas el analisis se va realizar para un rago salarial de 1'160.000 pesos (salario minimo) a 42'412.000 pesos (1000 UVT)




[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JoseRZapata/covid_plots/blob/main/notebooks/Covid19_Visualizacion_es.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/JoseRZapata/covid_plots/HEAD?labpath=notebooks%2FCovid19_Visualizacion_es.ipynb)  [![nbviewer](https://img.shields.io/badge/render-nbviewer-orange.svg)](https://nbviewer.jupyter.org/github/JoseRZapata/covid_plots/blob/main/notebooks/Covid19_Visualizacion_es.ipynb)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import plotly.express as px

#formato de numeros en tipo dinero
pd.set_option('display.float_format', lambda x: '{:,.0f}'.format(x))

In [2]:
#print pandas, px an numpy version
print('pandas version: ', pd.__version__)
print('numpy version: ', np.__version__)

pandas version:  2.1.3
numpy version:  1.26.2


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JoseRZapata/covid_plots/blob/main/notebooks/Covid19_Visualizacion_es.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/JoseRZapata/covid_plots/HEAD?labpath=notebooks%2FCovid19_Visualizacion_es.ipynb)  [![nbviewer](https://img.shields.io/badge/render-nbviewer-orange.svg)](https://nbviewer.jupyter.org/github/JoseRZapata/covid_plots/blob/main/notebooks/Covid19_Visualizacion_es.ipynb)

## Grafica de Salario Mensual

In [52]:
# constantes globales
uvt = 42_412
salario_minimo = 1_160_000
subsido_transporte = 140_606
# salario integral = 10 salarios minimos + 30% prestaciones
salario_integral = salario_minimo*(10)*(1 + 0.3)
salario_inte_uvt = salario_integral/uvt
print(f'Salario integral actual: ${salario_integral:,.0f}')

Salario integral actual: $15,080,000


En base a la tabla de rangos de retencion en la fuente presentado anteriormente,
voy a generar los rangos de ingresos mensuales.

Lo valores empezaran desde el salario minimo legal vigente en Colombia para el 2023 que es de 1'160.000 pesos colombianos.
Equivalente a 27.351 UVT (Unidad de Valor Tributario) para el 2023.

In [4]:
#rangos en uvt
rango = np.arange(28, 1001)

#convertir rango a tipo float
rango = rango.astype(float)

#agregar el valor de salario_minimo/uvt = 27.35
rango = np.insert(rango, 0, salario_minimo/uvt)

#agregar el valor de salario_integral_uvt
index_integral = (np.argwhere(rango == int(salario_inte_uvt))
                  .flatten()[0])
rango = np.insert(rango, index_integral+1, salario_inte_uvt)

salario_col = pd.DataFrame(rango, columns=['n_uvt'])
salario_col['salario_pesos'] = ((salario_col['n_uvt'] * uvt)
                                .astype(int))

In [5]:
# funcion para calcular el pago de salud y pension
def pago_salud_pension(salario_pesos: int,
                       salario_integral: int = salario_integral
                       )-> int:
    """ 
    Calcula el pago de salud y pension para un salario dado en pesos,
    es igual al 4% del salario si es menor al salario integral,
    si es mayor al salario integral es igual al 4% del 70% del salario
    
    Args:
        salario_pesos (int): salario en pesos
        salario_integral (int): salario integral en pesos
    Returns:
        int: pago de salud y pension       
    
    """
    
    if salario_pesos < salario_integral:
        if salario_pesos < salario_minimo:
            aportes_salud_pension = salario_minimo * 0.04
        else:
            aportes_salud_pension =  salario_pesos * 0.04
        
    else:
        # si el salario es integral el aporte es sobre el 70% del salario
        aportes_salud_pension = salario_pesos *.7 * 0.04
         
    return aportes_salud_pension

In [6]:
def pago_solidaridad(salario_pesos: int,
                     salario_minimo: int = salario_minimo,
                     salario_integral: int = salario_integral
                     )-> int:
    """ 
    Calcula el pago de fondo de solidaridad pensional para un salario dado en pesos,
    teneiendo en cuenta el ingreso base de cotizacion (IBC)
       
    Args:
        salario_pesos (int): salario en pesos
        salario_minimo (int): salario minimo en pesos
        salario_integral (int): salario integral en pesos
    Returns:
        int: pago de salud y pension       
    """
    if salario_pesos < salario_integral:
        ingreso_base_cotizacion = salario_pesos
    else:
        #si el salario es igual o mayor que el salario integral
        # el IBC es el 70% del salario    
        ingreso_base_cotizacion = salario_pesos*0.7
    
    if ingreso_base_cotizacion <= 4 * salario_minimo:
        aporte_solidaridad = 0
    elif  4 * salario_minimo <= ingreso_base_cotizacion <= 16 * salario_minimo:
        aporte_solidaridad = ingreso_base_cotizacion * 0.01
    elif 16 * salario_minimo < ingreso_base_cotizacion <= 17 * salario_minimo:
        aporte_solidaridad = ingreso_base_cotizacion * 0.012
    elif 17 * salario_minimo < ingreso_base_cotizacion <= 18 * salario_minimo:
        aporte_solidaridad = ingreso_base_cotizacion * 0.014
    elif 18 * salario_minimo < ingreso_base_cotizacion <= 19 * salario_minimo:
        aporte_solidaridad = ingreso_base_cotizacion * 0.016
    elif 19 * salario_minimo < ingreso_base_cotizacion <= 20 * salario_minimo:
        aporte_solidaridad = ingreso_base_cotizacion * 0.018
    else:
        aporte_solidaridad = ingreso_base_cotizacion * 0.02    
        
    return aporte_solidaridad

In [7]:
salario_col['pago_salud'] = (salario_col['salario_pesos']
                                   .apply(pago_salud_pension,
                                          salario_integral=salario_integral))
salario_col['pago_pension'] = salario_col['pago_salud']

salario_col['pago_solidaridad'] = (salario_col['salario_pesos']
                                   .apply(pago_solidaridad,
                                          salario_minimo=salario_minimo,
                                          salario_integral=salario_integral))

# Auxilio de transporte si el salario es menor a 2 salarios minimos,
# de lo contrario es 0

salario_col['aux_transporte'] = np.where(salario_col['salario_pesos'] <= 2*salario_minimo,
                                            subsido_transporte, 0)

In [8]:
def retencion_fuente(n_uvt:float,
                     uvt:int )-> float:
    """
    Calcula la retencion en la fuente para un salario en uvt
    
    Args:
        n_uvt (float): salario en uvt
        uvt   (int)  : valor de un uvt
    
    Returns:
        float: valor de la retencion en la fuente en pesos       
    """
    # la tupla se compone de:
    # limite inferior, limite superior, porcentaje, uvt que se suman
    rangos = [(0,   95,  0, 0),
              (95,  150, 0.19, 0),
              (150, 360, 0.28, 10),
              (360, 640, 0.33, 69),
              (640, 945, 0.35, 162),
              (945, 2300, 0.37, 268)]
    retencion = 0
    for limite_inf, limite_sup, porcentaje, base in rangos:
        if (limite_inf < n_uvt <= limite_sup):
            retencion = (n_uvt - limite_inf) * porcentaje + base
            break
    return retencion * uvt        

In [9]:
# usar metodo .assing para crear una nueva columna
# para hacer copia de los datos

salario_col = (salario_col
               .assign(base_retencion = salario_col['salario_pesos']
                                        - salario_col['pago_salud'] 
                                        - salario_col['pago_pension']
                                        - salario_col['pago_solidaridad']))

salario_col = (salario_col
               .assign(base_retencion_uvt = salario_col['base_retencion']/uvt))

salario_col = (salario_col
               .assign(retencion_fuente = salario_col['base_retencion_uvt']
                                          .apply(retencion_fuente,
                                                 uvt=uvt)))
salario_col = (salario_col
               .assign(deduccion_mensual = salario_col['pago_salud'] 
                                           + salario_col['pago_pension']
                                           + salario_col['pago_solidaridad'] 
                                           + salario_col['retencion_fuente']))

# Dinero entregado al empleado
salario_col = (salario_col
               .assign(salario_neto_mes = salario_col['salario_pesos']
                                         + salario_col['aux_transporte']
                                         - salario_col['deduccion_mensual']))

# salario integral en uvt
salario_col['salario_anual_neto'] = salario_integral/uvt

In [31]:
# grafiar salario_pesos, salario_neto vs n_uvt en plotly
fig = px.line(salario_col, x='n_uvt', y=['salario_pesos',
                                         'deduccion_mensual',
                                         'salario_neto_mes',],
              title='Salario y Salario neto (Luego de las deducciones)'
                    ' [Mensual sin prima]',
              labels={'n_uvt': 'Salario en UVT',
                      'value': 'Salario en pesos',
                      'variable': 'Tipo de salario'})
fig.update_traces(line=dict(width=3))

#linea vertical en salario integral alpha=0.5
fig.add_vline(x=salario_inte_uvt, line_width=3, line_dash="dot",
              line_color="black", opacity=0.8,
              annotation_text="<-Salario Integral uvt",
              annotation_position="right")

#area entre n_uvt = 96 y n_uvt = 150
fig.add_vrect(x0=96, x1=150, line_width=0, fillcolor="cyan", opacity=0.2,
              annotation_text="Retencion 19%", annotation_position="left")

#area entre n_uvt = 151 y n_uvt = 360
fig.add_vrect(x0=151, x1=360, line_width=0, fillcolor="yellow", opacity=0.2,
              annotation_text="Retencion 28%", annotation_position="top")
#area entre n_uvt = 361 y n_uvt = 640
fig.add_vrect(x0=361, x1=640, line_width=0, fillcolor="blue", opacity=0.2,
              annotation_text="Retencion 33%", annotation_position="top")
#area entre n_uvt = 641 y n_uvt = 945
fig.add_vrect(x0=641, x1=945, line_width=0, fillcolor="green", opacity=0.2,
              annotation_text="Retencion 35%", annotation_position="top")
#area entre n_uvt = 946 y n_uvt = 1000
fig.add_vrect(x0=946, x1=1000, line_width=0, fillcolor="purple", opacity=0.2,
              annotation_text="Retencion 37%")

fig.show()

## Grafica Salario Anual


Teniendo en cuenta lo previsto en el artículo 307 del Código Sustantivo del Trabajo, según el cual la prima anual no es salario ni se computa como factor salarial en ningún caso, la misma no se debe considerar para efectos de calcular la base de cotización de aportes al sistema general de seguridad social en salud o pensiones, indicó el Ministerio de Salud. 


In [12]:
salario_col['prima'] = (salario_col['salario_pesos'] 
                        + salario_col['aux_transporte'])/2

# si salario_pesos es mayor de salario_integral prima es 0
salario_col['prima'] = np.where(salario_col['salario_pesos'] >= salario_integral,
                                0, salario_col['prima'])

# las cesantias son igual a un salario completo
salario_col['cesantias'] = salario_col['prima']*2


In [13]:
salario_col['salario_anual'] = (salario_col['salario_pesos'] * 12 
                                + salario_col['prima']*2 
                                + salario_col['aux_transporte']*12
                                + salario_col['cesantias'])

salario_col['deduccion_anual'] = salario_col['deduccion_mensual'] * 12


#falta hacer el salario_anual_neto tener en cuenta
# la retencion en la fuente de los meses que se recibe la prima
salario_col['salario_anual_neto'] = (salario_col['salario_anual']
                                     - salario_col['deduccion_anual'])

In [14]:
# salario_integral_anual_neto 
int_salario_inte_uvt = int(salario_inte_uvt)

salario_integral_anual_neto = (salario_col
                               .query("n_uvt == @int_salario_inte_uvt")
                               ['salario_anual_neto']).values[0]

# valor de uvt en el cual luego de pasar el salario integral
# se empieza a ganar mas dinero anualmente que recibiendo un poco menos
# que el salario integral
n_uvt_nivel = (salario_col
               .query("salario_anual_neto > @salario_integral_anual_neto")
               ['n_uvt'].iloc[0])

In [58]:
import plotly.graph_objects as go
# grafiar salario_pesos, salario_neto vs n_uvt en plotly
fig = px.line(salario_col, x='n_uvt', y=['salario_anual', 'salario_anual_neto'],
              title='Salario y Salario neto (Luego de las deducciones)'
                    ' [Anual con prima y cesantias]',
              labels={'n_uvt': 'Salario en UVT',
                      'value': 'Salario en pesos',
                      'variable': 'Tipo de salario'})

fig.update_traces(line=dict(width=3))

#linea horizontal en salario integral alpha=0.5
fig.add_hline(y=salario_integral_anual_neto, line_width=2, line_dash="dot",
              line_color="black", opacity=0.8,
              annotation_text="Salario Integral $",
              annotation_position="bottom right")

#linea vertical en salario integral
fig.add_vline(x=salario_inte_uvt, line_width=3, line_dash="dot",
              line_color="black", opacity=0.8,
              annotation_text="<-Salario Integral uvt",
              annotation_position="right")

#area entre n_uvt = 96 y n_uvt = 150
fig.add_vrect(x0=96, x1=150, line_width=0, fillcolor="cyan", opacity=0.2,
              annotation_text="Retencion 19%", annotation_position="left")

#area entre n_uvt = 151 y n_uvt = 360
fig.add_vrect(x0=151, x1=360, line_width=0, fillcolor="yellow", opacity=0.2,
              annotation_text="Retencion 28%", annotation_position="top")
#area entre n_uvt = 361 y n_uvt = 640
fig.add_vrect(x0=361, x1=640, line_width=0, fillcolor="blue", opacity=0.2,
              annotation_text="Retencion 33%", annotation_position="top")
#area entre n_uvt = 641 y n_uvt = 945
fig.add_vrect(x0=641, x1=945, line_width=0, fillcolor="green", opacity=0.2,
              annotation_text="Retencion 35%", annotation_position="top")
#area entre n_uvt = 946 y n_uvt = 1000
fig.add_vrect(x0=946, x1=1000, line_width=0, fillcolor="purple", opacity=0.2,
              annotation_text="Retencion 37%")
# rectangulo entre n_uvt = salario_inte_uvt y n_uvt= n_uvt_nivel
fig.add_vrect(x0=salario_inte_uvt, x1=n_uvt_nivel, line_width=2, fillcolor="red",
              opacity=0.4, annotation_text="Rango donde <br>" 
                                           "Se disminuye <br>el salario<br> anual neto",
              annotation_position="bottom left")

fig.add_trace(go.Scatter(
    x=[n_uvt_nivel],
    y=[salario_integral_anual_neto],
    mode="markers",
    marker=dict(size=10, color="Green"),
    name="Salario Integral con ganancia"
))

fig.add_trace(go.Scatter(
    x=[int(salario_inte_uvt)],
    y=[salario_integral_anual_neto],
    mode="markers",
    marker=dict(size=10, color="Green"),
    name="Antes de Salario Integral"
))


fig.show()

El salario integral incluye en su cuantía todos los conceptos salariales y prestacionales como:
- La prima de servicios 
- El auxilio de cesantías
- Pago de dominicales y festivos laborados
- El pago de horas extra
- Los recargos nocturnos
- Las primas extralegales que pague la empresa

por este motivo al no tener en cuenta la prima de servicios y cesantias, hay un rango del salario integral neto anual donde se percibe menos dinero que un asalariado. Ademas el porcentaje de retencion en la fuente es mayor para el salario integral. En base a estos resultados

In [54]:
print('el salario integral mensual donde se empieza\n'
      f'a ganar mas anualmente es: ${n_uvt_nivel*uvt:,.0f}')

el salario integral mensual donde se empieza
a ganar mas anualmente es: $18,534,044


Es importante anotar que las empresas en multiples situaciones agregan beneficios adicionales a los empleados que tienen salario integral con el fin de compensar la perdida de ingresos.

# Refencias

- https://blog.alegra.com/retencion-en-la-fuente/
- https://www.consultorcontable.com/retenci%C3%B3n-salarios/
- https://loggro.com/blog/articulo/conozca-como-quedo-el-salario-minimo-y-salario-integral-en-2023/
- https://loggro.com/blog/articulo/como-calcular-el-salario-integral-loggro-nomina/
- https://www.larepublica.co/economia/asi-quedaria-el-aporte-al-fondo-de-solidaridad-pensional-con-la-ponencia-radicada-3630684

